# UNDER CONSTRUCTION !

# Run the code at your own risk !

In [63]:
import pandas as pd
from pandas.testing import assert_frame_equal
import numpy as np
import csv 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split

In [64]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', 5000)

In [65]:
# Read csv
asthma_df = pd.read_csv('astma.csv')
healthy_df = pd.read_csv("healthy_parsed.csv")

# Preprocessing

## Removing columns with too much missing data

In this section, we will remove the columns having too many missing values. First for the healthy kids, then for the sick ones.

#### Technique
About technique 2 Matthijs wrote this:

Is it possible to do your analyses with both options to see whether it impacts the performance of your models? If not, I would just use the ones marked ‘2’ anyway to include as much data as possible.


In [66]:
asthma_df['Technique'].value_counts()
asthma_df = asthma_df.drop(asthma_df[asthma_df['Technique'] == 1].index).reset_index().drop('index', axis=1)
# IF WE DELETE THE WRONG TECHNIQUE HERE, ANY dadaset.loc[i] WILL GIVE AN ERROR AT THE FIRST MISSING ROW
# We need to reset the indicies to avoid this

# Alligning column names

In [67]:
# Renaming columns (allignment)
asthma_df = asthma_df.rename(columns={'Gender': 'sex'})
asthma_df = asthma_df.rename(columns={'Weight': 'weight'})
asthma_df = asthma_df.rename(columns={'Height': 'height'})

# What grade they are in
healthy_df = healthy_df.rename(columns={'school_year_final': 'school_year'})
asthma_df = asthma_df.rename(columns={'school_year_edit': 'school_year'})

# Wheter they do a sport
asthma_df = asthma_df.rename(columns={'Sports': 'sportsyesno'})

# Note there are 2 urbanisations
asthma_df = asthma_df.rename(columns={'urbanisation2': 'urbanisation'})

# pedsql
asthma_df = asthma_df.rename(columns={'BASELINE_PedsQL_score': 'PedsQL_score_baseline'})

# Note healthy_df doesn't have all wear variables (it only has the grped vars)
# Asthma_df doesnt have wear05H lets compute it based on the other wear vars
asthma_df['wear05H'] = (asthma_df.loc[:, 'wear00':'wear05'] == 1).mean(axis=1) * 100

# No temperature in asthma_df but is in healthy_df?
# Drop the col
healthy_df = healthy_df.drop('BODY_TEMPERATURE_DEG_C', axis=1)

# No blood preassure in asthma_df but is in healthy_df?
# drop in healthy
healthy_df = healthy_df.drop('DIASTOLIC_BLOOD_PRESSURE_MMHG', axis=1)
healthy_df = healthy_df.drop('SYSTOLIC_BLOOD_PRESSURE_MMHG', axis=1)

# Note that WEIGHT_KG is a daily meassurement in healthy_df
# in asthma_df this meassurement is only taken at begin and end of the study period
# Also missing data is 87% lets drop this col
healthy_df = healthy_df.drop('WEIGHT_KG', axis=1)

# I am assuming these columns are the same (bedtimeReport and waketimeReport not in legend))
asthma_df = asthma_df.rename(columns={'bedtimeReport': 'sleeptime'})
asthma_df = asthma_df.rename(columns={'waketimeReport': 'waketime'})

# Create a new column with the hourly categories starting from 0 to 23
hour_mapping = {i: i for i in range(24)}

# Converting to datetime
healthy_df['sleeptime'] = pd.to_datetime(healthy_df['sleeptime'])
asthma_df['sleeptime'] = pd.to_datetime(asthma_df['sleeptime'])
healthy_df['waketime'] = pd.to_datetime(healthy_df['waketime'])
asthma_df['waketime'] = pd.to_datetime(asthma_df['waketime'])

# Setting hourly value
healthy_df['sleeptime'] = healthy_df['sleeptime'].dt.hour
asthma_df['sleeptime'] = asthma_df['sleeptime'].dt.hour
healthy_df['waketime'] = healthy_df['waketime'].dt.hour
asthma_df['waketime'] = asthma_df['waketime'].dt.hour

# predicted_fvc_best, predicted_fev1_best, predicted_fev1_ratio_best
# All of these not in asthma
healthy_df = healthy_df.drop('predicted_fvc_best', axis=1)
healthy_df = healthy_df.drop('predicted_fev1_best', axis=1)
healthy_df = healthy_df.drop('predicted_fev1_ratio_best', axis=1)

# Screentime, values need to be alligned 
# Alignment dictionary
alignment_dict = {
    '0': '0',
    'D. 2 uur tot 4 uur': '2-4 hours',
    'A. 0 tot 30 minuten': '0-30 min',
    'C. 1 uur tot 2 uur': '1-2 hours',
    'B. 30 tot 60 minuten': '0.5-1 hours',
    'E. Meer dan 4 uur': '> 4 hours'
}

# Rename values in the first dataframe based on the alignment dictionary
asthma_df['screentime'] = asthma_df['screentime'].replace(alignment_dict)

# Dealing with missing data


## Issue with HR columns

In the asthma dataset there seems to be a problem with HR00 to HR23. Sometimes, the heart rate goes above a million sometimes. It seems like the first couple digits correspond with the expected heart rate.

In [68]:
# In the asthma dataset there seems to be a problem with HR00 to HR23
# The heart rate goes above a million sometimes
# It seems like the first couple digits correspond with the expected heart rate.
# 2 cases: HR below 100 and HR above 100

# Loop through each column in the dataframe
for col in asthma_df.loc[:, "HR00":"HR23"]:
    for i in range(len(asthma_df[col])):
        if (asthma_df.loc[i, col] > 1000):
            # Take first 2 digits of float
            first_2_digits = str(asthma_df.loc[i, col])[:2]
            temp_number = float(first_2_digits)
            # If heart rate lower than 20 it means it should be greater than 100 (assuming heart rates < 200)
            if temp_number < 20:
                val = str(asthma_df.loc[i, col])[:3] + '.' + str(asthma_df.loc[i, col])[3]
                asthma_df.loc[i, col] = float(val)
            else:
                val = str(asthma_df.loc[i, col])[:2] + '.' + str(asthma_df.loc[i, col])[2]
                asthma_df.loc[i, col] = float(val)
            pass

In [69]:
# Looking at the possible values of different columns in the healthy and sick database.
print("Let's allign these values: ")
print(healthy_df['weekday'].unique())
print(asthma_df['weekday'].unique())
print()
print(healthy_df['dayType'].unique())
print(asthma_df['dayType'].unique())
print()
print(healthy_df['school_yes_no'].unique())
print(asthma_df['school_yes_no'].unique())
print()
print(healthy_df['sex'].unique())
print(asthma_df['sex'].unique())
print()
print(healthy_df['school_year'].unique())
print(asthma_df['school_year'].unique())
print()
print(healthy_df['sportsyesno'].unique())
print(asthma_df['sportsyesno'].unique())
print()
print(healthy_df['urbanisation'].unique())
print(asthma_df['urbanisation'].unique())
print()
print(healthy_df['grade_fev1'].unique())
print(asthma_df['grade_fev1'].unique())
print()
print(healthy_df['grade_fvc'].unique())
print(asthma_df['grade_fvc'].unique())
print()
print(healthy_df['fev1_ratio_best'].unique())
print(asthma_df['fev1_ratio_best'].unique())
print("I am not sure why there are only true or nans here???")
print()
print(healthy_df['screentime'].unique())
print(asthma_df['screentime'].unique()) 

Let's allign these values: 
['Thu' 'Fri' 'Sat' 'Sun' 'Mon' 'Tue' 'Wed']
['Tue' 'Wed' 'Thu' 'Fri' 'Sat' 'Sun' 'Mon']

['holiday' 'school' 'weekend']
['school' 'weekend' 'holiday']

[nan 'Neither' 'School' 'Day Care']
[nan 'Ja' 'Nee']

['Female' 'Male']
['Female' 'Male' nan]

['Day care' 'Primary school' 'Secondary school' 'Vocational education']
['Secondary school year 4' 'Primary school year 5'
 'Secondary school year 1' 'Primary school year 7' 'Primary school year 4'
 'Primary school year 8' 'Secondary school year 3' 'Primary school year 3'
 'Secondary school year 2' 'Primary school year 6' nan]

['No' 'Yes']
['Yes' 'No' nan]

['Extremely urbanised' 'Moderately urbanised' 'Very urbanised'
 'Little urbanised']
['Extremely urbanised' 'Not extremely urbanised' nan]

[nan 'E' 'A' 'C' 'U' 'B' 'D' 'F']
['D' 'A' 'B' 'E' 'C' 'U' nan]

[nan 'E' 'A' 'C' 'U' 'B' 'D' 'F']
['B' 'A' 'C' 'E' 'D' 'U' nan]

[nan True]
[0.73669555 0.69211898 0.74384237 ... 0.83669886 0.83624273 0.80671937]
I am not sur

In [70]:
# school_yes_no
# for healthy: school, daycare or neither
# for asthma: yes or no (school)
# Decision: school and daycare are the same class
# note that nan will be also be a category in the one-hot encoding (unknown)
healthy_df['school_yes_no'] = healthy_df['school_yes_no'].replace({'Neither': 'no', 'Day Care': 'yes', 'School': 'yes'})
asthma_df['school_yes_no'] = asthma_df['school_yes_no'].replace({'Nee': 'no', 'Ja': 'yes'})

# sex
# it doesnt make sense to one hot encode nan for one subject 
# this won't improve classification, we will have to assume a gender 
# Women are more likely to classify as non-binary --> assume female
# https://www.pewresearch.org/social-trends/2022/06/28/americans-complex-views-on-gender-identity-and-transgender-issues/
asthma_df['sex'] = asthma_df['sex'].fillna('Female')

# school year
# I think it will be best to drop this its likely to different in each data set 
healthy_df = healthy_df.drop('school_year', axis=1)
asthma_df = asthma_df.drop('school_year', axis=1)

# sports_yes_no
# A couple subjects didn't fill this in, I think it is a fair assumption to made that 
# if they didn't fill it in than they didn't do sports
asthma_df['sportsyesno'] = asthma_df['sportsyesno'].fillna('No')

# Urbanization
# Decision: Extremely and very will be merged to extremely urbanized
# moderately and little urbanized will be merged to not extremely urbanized
healthy_df['urbanisation'].replace({'Very urbanised': 'Extremely urbanised',
                                    'Moderately urbanised': 'Not extremely urbanised', 
                                    'Little urbanised': 'Not extremely urbanised'}, 
                                   inplace=True)

In [71]:
# one-hot encoding
one_hot_asthma = pd.get_dummies(asthma_df[['weekday', 'dayType', 'school_yes_no', 'sex', 'sportsyesno', 'urbanisation',
                                          'grade_fev1', 'grade_fvc', 'screentime']])
one_hot_healthy = pd.get_dummies(healthy_df[['weekday', 'dayType', 'school_yes_no', 'sex', 'sportsyesno', 'urbanisation',
                                          'grade_fev1', 'grade_fvc', 'screentime']])

# Merging
asthma_df = pd.concat([asthma_df, one_hot_asthma], axis=1)
healthy_df = pd.concat([healthy_df, one_hot_healthy], axis=1)

# Make sure to drop old columns (they are replaced with the hot-encoded cols)
healthy_df = healthy_df.drop(columns=['weekday', 'dayType', 'school_yes_no', 'sex', 'sportsyesno', 'urbanisation',
                                          'grade_fev1', 'grade_fvc', 'screentime'])
asthma_df = asthma_df.drop(columns=['weekday', 'dayType', 'school_yes_no', 'sex', 'sportsyesno', 'urbanisation',
                                          'grade_fev1', 'grade_fvc', 'screentime'])


lista = ['AllergicRhin', 'AtopicAsthma', 'CF_Asthma', 'DailyActiv', 'ExerciseSymp', 'Family_his', 'ICS', 'LABA', 'Leukotri_Mod',
         'Monoclonals', 'Oral_Steroid', 'OtherClass', 'PancrInsuffi', 'PetsHome', 'SABA', 'SmokeHome', 'CONDITION']

one_hot_asthma = pd.get_dummies(asthma_df[lista])

asthma_df = pd.concat([asthma_df, one_hot_asthma], axis=1)

asthma_df = asthma_df.drop(columns=lista)

In [72]:
# Just an index we dont need it
healthy_df = healthy_df.drop('Index', axis=1)
asthma_df = asthma_df.drop('Unnamed: 0', axis=1)

# Subjects number are not relevant for predicting
#healthy_df = healthy_df.drop('SubjectNr', axis=1)
#asthma_df = asthma_df.drop('SubjectNr', axis=1)

# We only have ethnicity in healthy_df
healthy_df = healthy_df.drop('ethnicity', axis=1)

# Modeling

### MissForest Test

In [73]:
from missingpy import MissForest
imputer = MissForest()

In [74]:
d = {'col1': [1, 1,0,1,0,0],
     'col2': [7.2500, 71.2833, 7.9250, 53.1000, 8.0500, 8.4583],
     'col3': [22.0, 38.0, 26.0, 35.0, 35.0, np.nan]}

df = pd.DataFrame(data=d)
df

,col1,col2,col3
0,1,7.2500,22.0
1,1,71.2833,38.0
2,0,7.9250,26.0
3,1,53.1000,35.0
4,0,8.0500,35.0
5,0,8.4583,NaN


In [75]:
imputer = MissForest()
imputer.fit_transform(df)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

array([[ 1.    ,  7.25  , 22.    ],
       [ 1.    , 71.2833, 38.    ],
       [ 0.    ,  7.925 , 26.    ],
       [ 1.    , 53.1   , 35.    ],
       [ 0.    ,  8.05  , 35.    ],
       [ 0.    ,  8.4583, 31.12  ]])

### MissForest apply

#### First try

In [76]:
asthma_df.head()

,SubjectNr,DayNo,stepsTotalDaily,stepsTotalDetailed,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HrAvgWake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,HRAvgSleep,HRMinSleep,HRMaxSleep,activity_score_parent,activity_score_child,sleep_score_child,sleeptime,waketime,fvc_best,fev1_best,pef_best,Technique,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,ACD1,ACD2,ACD3,ACD4,ACD5,ACD6,ACD6_use,ACD6score,BASELINE_ACQ6_score,EOS_ACQ6_score,BASELINE_physical_tot,EOS_physical_tot,BASELINE_emotional_tot,EOS_emotional_tot,BASELINE_school_tot,EOS_school_tot,BASELINE_bodyimage_tot,EOS_bodyimage_tot,BASELINE_treatmentburden_tot,EOS_treatmentburden_tot,BASELINE_health_tot,EOS_health_tot,BASELINE_weight_tot,EOS_weight_tot,BASELINE_respiratory_tot,EOS_respiratory_tot,BASELINE_digestion_tot,EOS_digestion_tot,BASELINE_PAQLQ_total,EOS_PAQLQ_total,BASELINE_PAQLQ_activity,EOS_PAQLQ_activity,BASELINE_PAQLQ_symptoms,EOS_PAQLQ_symptoms,BASELINE_PAQLQ_emotion,EOS_PAQLQ_emotion,PedsQL_score_baseline,EOS_PedsQL_score,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,pollenTotal,NO,NO2,Nox,O3,Ox,PM10,PM25,Symptomscore,Admission_LY,Age,AgeDiagnosis,BirthMonth,BirthYear,BMI,BMI_SDS,DayCare,Dis_Chronic,Eosinophils,height,Hosp_treatme,Hospital_Trt,Pseudomonas,SchoolYear,Serum_IgE,Vaccination,weight,Urbanisation,totalSleepDuration,wear00,wear01,wear02,wear03,wear04,wear05,wear06,wear07,wear08,wear09,wear10,wear11,wear12,wear13,wear14,wear15,wear16,wear17,wear18,wear19,wear20,wear21,wear22,wear23,wear24H,wear16H,steps15_19,week,steps_hour_max,AVGHR_daily,AVGHR_wake,AVGHR_sleep,EventDay,wear05H,weekday_Fri,weekday_Mon,weekday_Sat,weekday_Sun,weekday_Thu,weekday_Tue,weekday_Wed,dayType_holiday,dayType_school,dayType_weekend,school_yes_no_no,school_yes_no_yes,sex_Female,sex_Male,sportsyesno_No,sportsyesno_Yes,urbanisation_Extremely urbanised,urbanisation_Not extremely urbanised,grade_fev1_A,grade_fev1_B,grade_fev1_C,grade_fev1_D,grade_fev1_E,grade_fev1_U,grade_fvc_A,grade_fvc_B,grade_fvc_C,grade_fvc_D,grade_fvc_E,grade_fvc_U,screentime_0-30 min,screentime_0.5-1 hours,screentime_1-2 hours,screentime_2-4 hours,screentime_> 4 hours,AllergicRhin_No,AllergicRhin_Yes,AtopicAsthma_No,AtopicAsthma_Yes,CF_Asthma_Controlled asthma,CF_Asthma_Cystic Fibrosis,CF_Asthma_Uncontrolled asthma,DailyActiv_1-2x a week playing tennis,DailyActiv_2,DailyActiv_2x a week fitness,DailyActiv_M,DailyActiv_School,DailyActiv_School and sports,DailyActiv_School sports,DailyActiv_School.,DailyActiv_football bassball on regular basis,DailyActiv_guitar soccar school,DailyActiv_hockey and tennis 3x 1x,DailyActiv_ice skating skating tennis every day,DailyActiv_school,DailyActiv_school fitkids,DailyActiv_school sports,DailyActiv_school tennis,DailyActiv_school sports,DailyActiv_sports school,ExerciseSymp_No,ExerciseSymp_Yes,Family_his_No,Family_his_Yes,ICS_No,ICS_Yes,LABA_No,LABA_Yes,Leukotri_Mod_No,Leukotri_Mod_Yes,Monoclonals_No,Monoclonals_Yes,Oral_Steroid_No,Oral_Steroid_Yes,OtherClass_No,OtherClass_Yes,PancrInsuffi_No,PancrInsuffi_Yes,PetsHome_No,PetsHome_Yes,SABA_Yes,SmokeHome_No,SmokeHome_Yes,CONDITION_Controlled asthma,CONDITION_Cystic Fibrosis,CONDITION_Uncontrolled asthma
0,1771258,0,3723.0,3723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,137.0,NaN,NaN,605.0,1640.0,NaN,82.0,451.0,245.0,135.0,160.0,164.0,41.0,NaN,68.0,120.0,8.689362e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,NaN,NaN,NaN,89.8,83.4,87.8,91.5,93.30,77.1,78.6,660.0,16800.0,16440.0,1.0,75.0,61.0,93.0,NaN,NaN,NaN,NaN,NaN,3232.0,2381.0,4334.00,3.0,0.736696,0.726555,0.877857,0.822316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,2.666667e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.65

In [77]:
asthma_df.loc[:, 'HR00':'HR23'][2:10].isna().sum().sum()

0

In [78]:
asthma_df.loc[:, 'HR00':'HR23'][2:10]

,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23
2,79.1,73.1,80.30,78.0,76.1,74.6,78.1,98.50,103.4,100.5,109.6,100.6,105.00,109.0,86.50,88.6,102.8,78.2,85.2,94.0,100.00,89.25,96.5,82.00
3,77.1,79.8,72.60,67.0,80.2,76.0,73.6,79.00,98.5,103.5,106.4,95.5,106.20,98.2,100.00,84.3,86.8,97.2,74.0,107.0,82.40,96.10,90.8,78.20
4,83.3,77.6,75.10,76.2,72.6,75.8,73.8,68.00,74.8,65.4,94.4,85.7,77.25,86.5,78.00,76.6,90.0,89.1,92.2,110.6,77.40,81.10,76.5,82.00
5,67.3,68.7,73.75,64.8,62.3,61.8,69.0,75.30,69.0,76.0,100.6,93.0,70.50,89.3,81.00,93.3,98.7,107.6,101.6,86.1,86.30,91.10,89.5,74.75
6,71.8,75.0,74.30,64.0,69.5,73.3,72.0,78.25,103.0,103.0,93.8,93.1,89.50,89.7,96.10,84.5,86.5,95.1,88.6,96.5,86.00,79.00,80.5,75.00
7,82.2,70.1,68.50,64.4,69.8,63.0,89.0,99.50,94.7,101.4,97.6,99.5,94.10,88.6,84.60,101.7,95.8,94.8,89.5,101.0,100.25,88.30,81.1,70.80
8,73.5,98.0,62.00,66.7,63.8,64.3,61.5,60.00,88.8,117.8,99.0,90.5,83.20,81.0,91.60,98.6,86.6,89.0,106.6,131.5,98.70,82.10,69.2,72.20
9,88.3,65.1,63.80,63.1,58.5,68.0,74.8,87.10,93.1,104.7,99.8,95.6,98.40,93.1,98.25,80.3,91.5,86.4,94.8,104.2,86.10,86.00,74.0,78.50


In [79]:
hr_snap = asthma_df.loc[:, 'HR00':'HR23'][2:10]

x1 = np.random.randint(8)
y1 = np.random.randint(24)
hr_snap.iloc[x1][y1] = np.NaN

x2 = np.random.randint(8)
y2 = np.random.randint(24)
hr_snap.iloc[x2][y2] = np.NaN

x3 = np.random.randint(8)
y3 = np.random.randint(24)
hr_snap.iloc[x3][y3] = np.NaN

x4 = np.random.randint(8)
y4 = np.random.randint(24)
hr_snap.iloc[x4][y4] = np.NaN

x5 = np.random.randint(8)
y5 = np.random.randint(24)
hr_snap.iloc[x5][y5] = np.NaN

In [80]:
hr_snap

,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23
2,79.1,73.1,80.30,78.0,76.1,74.6,78.1,98.50,103.4,100.5,109.6,100.6,NaN,109.0,86.50,88.6,102.8,78.2,85.2,94.0,100.00,89.25,96.5,82.00
3,77.1,79.8,72.60,67.0,80.2,76.0,73.6,79.00,98.5,103.5,106.4,95.5,106.20,98.2,NaN,84.3,86.8,97.2,74.0,107.0,82.40,96.10,90.8,78.20
4,83.3,77.6,75.10,76.2,72.6,75.8,73.8,68.00,74.8,65.4,NaN,85.7,77.25,86.5,78.00,76.6,90.0,89.1,92.2,110.6,77.40,81.10,76.5,82.00
5,67.3,68.7,73.75,64.8,62.3,61.8,69.0,75.30,69.0,76.0,100.6,93.0,70.50,89.3,81.00,93.3,98.7,107.6,101.6,86.1,86.30,91.10,89.5,74.75
6,71.8,75.0,74.30,64.0,69.5,73.3,72.0,78.25,103.0,103.0,93.8,93.1,89.50,89.7,96.10,84.5,86.5,95.1,88.6,96.5,86.00,79.00,80.5,75.00
7,82.2,70.1,68.50,64.4,69.8,63.0,89.0,99.50,94.7,101.4,97.6,99.5,94.10,88.6,84.60,101.7,95.8,94.8,89.5,101.0,100.25,88.30,81.1,70.80
8,73.5,98.0,62.00,66.7,63.8,64.3,61.5,60.00,NaN,117.8,99.0,90.5,83.20,81.0,91.60,98.6,86.6,89.0,106.6,131.5,98.70,82.10,69.2,72.20
9,88.3,65.1,63.80,63.1,58.5,NaN,74.8,87.10,93.1,104.7,99.8,95.6,98.40,93.1,98.25,80.3,91.5,86.4,94.8,104.2,86.10,86.00,74.0,78.50


In [81]:
imputer = MissForest()
hr_snap_filled = imputer.fit_transform(hr_snap)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 0


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 1


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

...

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 3


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 4


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


In [82]:
hr_snap_unt = asthma_df.loc[:, 'HR00':'HR23'][2:10]
hr_snap_unt.iloc[x1][y1]

100.0

In [83]:
diffs = []

for (x,y) in [(x1,y1),(x2,y2),(x3,y3),(x4,y4),(x5,y5)]:
    diffs.append(hr_snap_unt.iloc[x][y] - hr_snap_filled[x][y])

In [84]:
diffs

[10.226999999999933,
 -6.908000000000044,
 4.120999999999967,
 10.963499999999968,
 -2.7290000000000845]

In [85]:
# it should be 94.7
hr_snap_filled[5][8]

94.7

Let's run for all the HR data for 1. patient

In [86]:
hr_subject1 = asthma_df.loc[asthma_df['SubjectNr'] == 1771258, 'HR00':'HR23']
hr_subject1.iloc[3][5] = np.NaN

In [87]:
hr_subject1_filled = imputer.fit_transform(hr_subject1)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 0


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 1


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 2


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

In [88]:
# it should be 61.8
hr_subject1_filled[3][5]

72.58000000000007

In [89]:
df = pd.DataFrame(data=hr_subject1_filled)

In [90]:
df.to_csv('df.csv')


#### Arthur's code

In [91]:
def extractChild(df):

    # Create a new database per children and save it into a list of all the same subject numbers.
    df_children = []    # List of all the children
    
    # Loop through all the subject numbers
    for subject in df['SubjectNr'].unique():
        df_children.append(df[df['SubjectNr'] == subject])
    
    return df_children


asthma_children = extractChild(asthma_df)

In [95]:
asthma_children[10].loc[:, "HR00":"HR23"]

,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23
287,78.3545,70.111,72.250,65.7570,67.859,69.8395,74.0080,92.964,102.804,90.6765,101.365,86.744,93.360,86.475,87.5600,95.3000,104.700,88.6,91.700,94.250,83.6000,70.000,69.800,57.300
288,64.6000,54.000,58.800,59.5000,60.000,74.2000,83.7000,90.000,103.800,96.8000,95.000,94.200,97.100,91.300,92.5000,82.1000,88.000,96.8,94.000,78.100,76.2000,68.400,61.500,68.700
289,59.6000,62.300,59.400,62.5000,66.400,69.7500,78.5000,79.100,82.500,95.5000,107.100,88.500,105.000,86.600,93.1000,95.5000,85.000,82.6,85.400,90.000,85.0000,80.000,70.200,60.800
290,54.8000,64.000,60.000,70.0000,67.000,76.7000,77.8000,84.000,87.100,72.4000,74.300,98.800,87.400,89.100,92.5000,92.1000,88.100,92.8,100.100,94.800,78.1000,79.300,78.100,79.800
291,78.0000,82.200,76.300,87.7500,81.600,71.5000,84.3000,83.000,91.600,79.7500,99.400,103.600,94.800,96.800,114.0000,114.1000,96.600,100.8,104.500,103.700,95.5000,95.800,95.800,93.100
292,86.2000,81.100,78.500,81.1000,73.800,73.2000,86.8000,95.400,98.300,84.0000,96.300,94.100,108.500,101.000,105.8000,100.4000,94.300,114.5,110.800,99.200,82.8000,76.400,74.500,71.500
293,75.0000,72.800,70.300,68.6000,72.000,68.3000,82.0000,98.100,98.300,95.3000,97.200,91.600,90.500,101.800,99.5000,88.0000,81.800,112.4,114.000,100.600,107.5000,90.000,91.700,89.000
294,78.7000,77.000,76.400,74.4000,70.000,76.5000,87.3000,82.400,100.100,97.2000,100.300,94.792,103.800,98.100,104.2500,100.7500,96.600,114.7,103.500,104.400,104.1000,97.000,89.700,87.600
295,73.7000,70.300,70.800,68.6000,64.600,66.5000,67.6000,81.300,89.800,83.1000,89.000,75.500,80.100,97.000,96.7985,90.2298,89.836,100.0,102.000,95.500,97.1000,78.400,71.600,67.100
296,69.1000,67.400,68.500,70.7000,69.100,70.0000,69.8000,85.600,100.500,86.8000,96.400,84.500,88.800,93.200,80.8000,90.4000,87.200,95.1,99.000,100.300,83.2500,90.600,91.643,83.254


In [62]:
from sklearn.impute import KNNImputer

for child in asthma_children:
    imputer = MissForest()          # n_neighbors=2
    child.loc[:, "HR00":"HR23"] = imputer.fit_transform(child.loc[:, "HR00":"HR23"])

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 0


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 1


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

...

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 2


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 3


KeyboardInterrupt: 

In [94]:
imputer = MissForest()
asthma_children[10].loc[:, "HR00":"HR23"] = imputer.fit_transform(asthma_children[10].loc[:, "HR00":"HR23"])

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 0


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 1


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

...

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 4


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 5


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

In [97]:
asthma_children[59].loc[:, "steps00":"steps23"]

,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23
1696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.0,836.0,228.0,138.0,958.0,511.0,557.0,1443.0,443.0,355.0,417.0,291.0,8.0,NaN,NaN,NaN
1697,NaN,NaN,NaN,NaN,NaN,NaN,55.0,15.0,1245.0,861.0,375.0,309.0,684.0,271.0,1802.0,439.0,390.0,212.0,334.0,NaN,NaN,NaN,NaN,NaN
1698,NaN,NaN,NaN,NaN,NaN,37.0,16.0,43.0,232.0,1419.0,320.0,808.0,807.0,474.0,380.0,174.0,126.0,73.0,59.0,NaN,NaN,NaN,22.0,58.0
1699,NaN,NaN,NaN,NaN,NaN,8.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,347.0,639.0,662.0,364.0,935.0,877.0,846.0,1009.0,364.0,223.0,164.0,162.0,NaN,NaN,NaN,NaN,NaN
1702,NaN,NaN,NaN,NaN,NaN,NaN,69.0,137.0,692.0,1065.0,793.0,508.0,1015.0,930.0,612.0,415.0,185.0,150.0,77.0,62.0,NaN,NaN,NaN,NaN
1703,NaN,NaN,NaN,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,339.0,639.0,1272.0,254.0,215.0,666.0,376.0,713.0,330.0,134.0,60.0,69.0,25.0,NaN,NaN,NaN,NaN
1705,NaN,NaN,NaN,NaN,NaN,7.0,66.0,NaN,203.0,300.0,419.0,364.0,90.0,117.0,268.0,115.0,172.0,426.0,67.0,88.0,NaN,NaN,NaN,NaN


In [103]:
for child in asthma_children:
    child.loc[:, "steps00":"steps06"] = child.loc[:, "steps00":"steps06"].fillna(0.0)
    child.loc[:, "steps12":"steps23"] = child.loc[:, "steps12":"steps23"].fillna(0.0)

/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_99619/1005753641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:, "steps00":"steps06"] = child.loc[:, "steps00":"steps06"].fillna(0.0)
/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_99619/1005753641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:, "steps12":"steps23"] = child.loc[:, "steps12":"steps23"].fillna(0.0)


In [104]:
asthma_children[59].loc[:, "steps00":"steps23"] = imputer.fit_transform(asthma_children[59].loc[:, "steps00":"steps23"])

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 0


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 1


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 2


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensembl

Iteration: 3


/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_99619/932706858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asthma_children[59].loc[:, "steps00":"steps23"] = imputer.fit_transform(asthma_children[59].loc[:, "steps00":"steps23"])


In [102]:
asthma_children[59][10:]

,SubjectNr,DayNo,stepsTotalDaily,stepsTotalDetailed,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HrAvgWake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,HRAvgSleep,HRMinSleep,HRMaxSleep,activity_score_parent,activity_score_child,sleep_score_child,sleeptime,waketime,fvc_best,fev1_best,pef_best,Technique,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,ACD1,ACD2,ACD3,ACD4,ACD5,ACD6,ACD6_use,ACD6score,BASELINE_ACQ6_score,EOS_ACQ6_score,BASELINE_physical_tot,EOS_physical_tot,BASELINE_emotional_tot,EOS_emotional_tot,BASELINE_school_tot,EOS_school_tot,BASELINE_bodyimage_tot,EOS_bodyimage_tot,BASELINE_treatmentburden_tot,EOS_treatmentburden_tot,BASELINE_health_tot,EOS_health_tot,BASELINE_weight_tot,EOS_weight_tot,BASELINE_respiratory_tot,EOS_respiratory_tot,BASELINE_digestion_tot,EOS_digestion_tot,BASELINE_PAQLQ_total,EOS_PAQLQ_total,BASELINE_PAQLQ_activity,EOS_PAQLQ_activity,BASELINE_PAQLQ_symptoms,EOS_PAQLQ_symptoms,BASELINE_PAQLQ_emotion,EOS_PAQLQ_emotion,PedsQL_score_baseline,EOS_PedsQL_score,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,pollenTotal,NO,NO2,Nox,O3,Ox,PM10,PM25,Symptomscore,Admission_LY,Age,AgeDiagnosis,BirthMonth,BirthYear,BMI,BMI_SDS,DayCare,Dis_Chronic,Eosinophils,height,Hosp_treatme,Hospital_Trt,Pseudomonas,SchoolYear,Serum_IgE,Vaccination,weight,Urbanisation,totalSleepDuration,wear00,wear01,wear02,wear03,wear04,wear05,wear06,wear07,wear08,wear09,wear10,wear11,wear12,wear13,wear14,wear15,wear16,wear17,wear18,wear19,wear20,wear21,wear22,wear23,wear24H,wear16H,steps15_19,week,steps_hour_max,AVGHR_daily,AVGHR_wake,AVGHR_sleep,EventDay,wear05H,weekday_Fri,weekday_Mon,weekday_Sat,weekday_Sun,weekday_Thu,weekday_Tue,weekday_Wed,dayType_holiday,dayType_school,dayType_weekend,school_yes_no_no,school_yes_no_yes,sex_Female,sex_Male,sportsyesno_No,sportsyesno_Yes,urbanisation_Extremely urbanised,urbanisation_Not extremely urbanised,grade_fev1_A,grade_fev1_B,grade_fev1_C,grade_fev1_D,grade_fev1_E,grade_fev1_U,grade_fvc_A,grade_fvc_B,grade_fvc_C,grade_fvc_D,grade_fvc_E,grade_fvc_U,screentime_0-30 min,screentime_0.5-1 hours,screentime_1-2 hours,screentime_2-4 hours,screentime_> 4 hours,AllergicRhin_No,AllergicRhin_Yes,AtopicAsthma_No,AtopicAsthma_Yes,CF_Asthma_Controlled asthma,CF_Asthma_Cystic Fibrosis,CF_Asthma_Uncontrolled asthma,DailyActiv_1-2x a week playing tennis,DailyActiv_2,DailyActiv_2x a week fitness,DailyActiv_M,DailyActiv_School,DailyActiv_School and sports,DailyActiv_School sports,DailyActiv_School.,DailyActiv_football bassball on regular basis,DailyActiv_guitar soccar school,DailyActiv_hockey and tennis 3x 1x,DailyActiv_ice skating skating tennis every day,DailyActiv_school,DailyActiv_school fitkids,DailyActiv_school sports,DailyActiv_school tennis,DailyActiv_school sports,DailyActiv_sports school,ExerciseSymp_No,ExerciseSymp_Yes,Family_his_No,Family_his_Yes,ICS_No,ICS_Yes,LABA_No,LABA_Yes,Leukotri_Mod_No,Leukotri_Mod_Yes,Monoclonals_No,Monoclonals_Yes,Oral_Steroid_No,Oral_Steroid_Yes,OtherClass_No,OtherClass_Yes,PancrInsuffi_No,PancrInsuffi_Yes,PetsHome_No,PetsHome_Yes,SABA_Yes,SmokeHome_No,SmokeHome_Yes,CONDITION_Controlled asthma,CONDITION_Cystic Fibrosis,CONDITION_Uncontrolled asthma
1706,5578891,10,445.0,445.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,445.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,77.0,98.0,8.646809e+09,82.0,81.0,83.1,83.6,86.3,85.10,91.6,112.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.6,10,7,6.3,53,7.4,3.1,37,0.4,4,4,0.0